In [1]:
from pyspark import SparkContext
from pyspark.sql import SparkSession

In [2]:
sc = SparkContext(appName="Join")
spark = SparkSession.builder\
        .master("local")\
        .appName('Join')\
        .getOrCreate()

22/06/10 08:14:46 WARN Utils: Your hostname, base resolves to a loopback address: 127.0.1.1; using 192.168.0.8 instead (on interface wlo1)
22/06/10 08:14:46 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/06/10 08:14:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/06/10 08:14:48 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [8]:
recschema = "idrec INT, datarec STRING, iddesp INT"
reclamacoes = spark.read.format("csv").load("/home/kiq/Stud/Spark/data/reclamacoes.csv",header=False,schema=recschema)
reclamacoes.show()

+-----+----------+------+
|idrec|   datarec|iddesp|
+-----+----------+------+
|    1|2020-09-12|     2|
|    2|2020-09-11|     2|
|    3|2020-10-05|     4|
|    4|2020-10-02|     5|
|    5|2020-12-06|     5|
|    6|2020-01-09|     5|
|    7|2020-01-05|     9|
+-----+----------+------+



In [15]:
despachantes = spark.read.format('parquet').load('/home/kiq/Stud/Spark/data/SparkOut/despparquet/part-00000-f5a5a5fd-6773-4d99-b5e1-4ad2ca065c37-c000.snappy.parquet')
despachantes.show()

+---+-------------------+------+-------------+------+----------+
| id|               nome|status|       cidade|vendas|      data|
+---+-------------------+------+-------------+------+----------+
|  1|   Carminda Pestana| Ativo|  Santa Maria|    23|2020-08-11|
|  2|    Deolinda Vilela| Ativo|Novo Hamburgo|    34|2020-03-05|
|  3|   Emídio Dornelles| Ativo| Porto Alegre|    34|2020-02-05|
|  4|Felisbela Dornelles| Ativo| Porto Alegre|    36|2020-02-05|
|  5|     Graça Ornellas| Ativo| Porto Alegre|    12|2020-02-05|
|  6|   Matilde Rebouças| Ativo| Porto Alegre|    22|2019-01-05|
|  7|    Noêmia   Orriça| Ativo|  Santa Maria|    45|2019-10-05|
|  8|      Roque Vásquez| Ativo| Porto Alegre|    65|2020-03-05|
|  9|      Uriel Queiroz| Ativo| Porto Alegre|    54|2018-05-05|
| 10|   Viviana Sequeira| Ativo| Porto Alegre|     0|2020-09-05|
+---+-------------------+------+-------------+------+----------+



In [10]:
spark.sql('show databases').show()

+---------+
|namespace|
+---------+
|  default|
+---------+



In [11]:
spark.sql('create database vendas')
spark.sql('use vendas').show()

++
||
++
++



In [13]:
reclamacoes.write.saveAsTable('reclamacoes')

In [16]:
despachantes.write.saveAsTable('despachantes')

In [17]:
spark.sql('show tables').show()

+---------+------------+-----------+
|namespace|   tableName|isTemporary|
+---------+------------+-----------+
|   vendas|despachantes|      false|
|   vendas| reclamacoes|      false|
+---------+------------+-----------+



In [21]:
spark.sql("select reclamacoes.*, despachantes.nome from despachantes \
            join reclamacoes on (despachantes.id == reclamacoes.iddesp)").show()

+-----+----------+------+-------------------+
|idrec|   datarec|iddesp|               nome|
+-----+----------+------+-------------------+
|    1|2020-09-12|     2|    Deolinda Vilela|
|    2|2020-09-11|     2|    Deolinda Vilela|
|    3|2020-10-05|     4|Felisbela Dornelles|
|    4|2020-10-02|     5|     Graça Ornellas|
|    5|2020-12-06|     5|     Graça Ornellas|
|    6|2020-01-09|     5|     Graça Ornellas|
|    7|2020-01-05|     9|      Uriel Queiroz|
+-----+----------+------+-------------------+



In [23]:
spark.sql("select reclamacoes.*, despachantes.nome from despachantes \
            left join reclamacoes on (despachantes.id == reclamacoes.iddesp)").show()

+-----+----------+------+-------------------+
|idrec|   datarec|iddesp|               nome|
+-----+----------+------+-------------------+
| null|      null|  null|   Carminda Pestana|
|    2|2020-09-11|     2|    Deolinda Vilela|
|    1|2020-09-12|     2|    Deolinda Vilela|
| null|      null|  null|   Emídio Dornelles|
|    3|2020-10-05|     4|Felisbela Dornelles|
|    6|2020-01-09|     5|     Graça Ornellas|
|    5|2020-12-06|     5|     Graça Ornellas|
|    4|2020-10-02|     5|     Graça Ornellas|
| null|      null|  null|   Matilde Rebouças|
| null|      null|  null|    Noêmia   Orriça|
| null|      null|  null|      Roque Vásquez|
|    7|2020-01-05|     9|      Uriel Queiroz|
| null|      null|  null|   Viviana Sequeira|
+-----+----------+------+-------------------+



In [25]:
despachantes.join(reclamacoes,on=despachantes.id == reclamacoes.iddesp, how='inner').select('idrec','datarec','iddesp','nome').show()

+-----+----------+------+-------------------+
|idrec|   datarec|iddesp|               nome|
+-----+----------+------+-------------------+
|    2|2020-09-11|     2|    Deolinda Vilela|
|    1|2020-09-12|     2|    Deolinda Vilela|
|    3|2020-10-05|     4|Felisbela Dornelles|
|    6|2020-01-09|     5|     Graça Ornellas|
|    5|2020-12-06|     5|     Graça Ornellas|
|    4|2020-10-02|     5|     Graça Ornellas|
|    7|2020-01-05|     9|      Uriel Queiroz|
+-----+----------+------+-------------------+

